In [33]:
import pandas as pd
import datetime as dt

In [72]:
df_2010_2011 = pd.read_excel("online_retail_II.xlsx" , sheet_name="Year 2010-2011")
df = df_2010_2011.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [73]:
df = df.dropna()
df['TotalPrice'] = df['Price']*df['Quantity']
today = dt.datetime(2011,12,11)

df = df[df['TotalPrice'] > 0]

df_rfm = df.groupby("Customer ID").agg({
    "InvoiceDate" : lambda date: (today - date.max()).days,
    "Invoice" : lambda num: len(num),
    "TotalPrice" : lambda TotalPrice:TotalPrice.sum()})

df_rfm.columns = ["Recency" , "Frequency" , "Monetary"]

df_rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346.0,326,1,77183.60
12347.0,3,182,4310.00
12348.0,76,31,1797.24
12349.0,19,73,1757.55
12350.0,311,17,334.40


In [74]:
df_rfm["RecencyScore"] = pd.qcut(df_rfm['Recency'], 5, labels = [5, 4, 3, 2, 1])
df_rfm["FrequencyScore"] = pd.qcut(df_rfm['Frequency'], 5, labels = [1,2,3,4,5])
df_rfm["MonetaryScore"] = pd.qcut(df_rfm['Monetary'], 5, labels = [1,2,3,4,5])

df_rfm["RFM_SCORE"] = df_rfm['RecencyScore'].astype(str) + df_rfm['FrequencyScore'].astype(str) + df_rfm['MonetaryScore'].astype(str)

df_rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12346.0,326,1,77183.60,1,1,5,115
12347.0,3,182,4310.00,5,5,5,555
12348.0,76,31,1797.24,2,3,4,234
12349.0,19,73,1757.55,4,4,4,444
12350.0,311,17,334.40,1,2,2,122


In [75]:
classification = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

df_rfm['Classification'] = df_rfm['RecencyScore'].astype(str) + df_rfm['FrequencyScore'].astype(str)
df_rfm['Classification'] = df_rfm['Classification'].replace(classification, regex=True)

df_rfm.head()


,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Classification
Customer ID,,,,,,,,
12346.0,326,1,77183.60,1,1,5,115,Hibernating
12347.0,3,182,4310.00,5,5,5,555,Champions
12348.0,76,31,1797.24,2,3,4,234,At Risk
12349.0,19,73,1757.55,4,4,4,444,Loyal Customers
12350.0,311,17,334.40,1,2,2,122,Hibernating


In [76]:
df_rfm[["Classification", "Recency","Frequency","Monetary"]].groupby("Classification").agg(["min","max","mean","sum","count"]).round()


Recency                           Frequency               \
                        min  max   mean     sum count       min   max   mean   
Classification                                                                 
About to Sleep           34   72   53.0   17128   322         1    29   16.0   
At Risk                  73  374  166.0   94702   569        30   119   57.0   
Can't Loose              73  373  144.0   12115    84       122   543  182.0   
Champions                 1   13    6.0    3879   606        59  7847  289.0   
Hibernating              73  374  210.0  225810  1074         1    29   14.0   
Loyal Customers          15   72   34.0   27947   817        59  1204  157.0   
Need Attention           34   72   53.0   10862   205        30    58   42.0   
New Customers             1   12    7.0     423    57         1    14    8.0   
Potential Loyalists       1   33   17.0    8201   492        15    58   35.0   
Promising                15   33   23.0    2625   112         1    14    8.0   

                                  Monetary                                     
                        sum count      min       max    mean        sum count  
Classification                                                                 
About to Sleep         5148   322      6.0    7331.0   451.0   145221.0   322  
At Risk               32353   569     67.0   11073.0   997.0   567246.0   569  
Can't Loose           15258    84    412.0   10254.0  2371.0   199139.0    84  
Champions            175133   606    190.0  280206.0  6961.0  4218315.0   606  
Hibernating           14591  1074      4.0   77184.0   537.0   576239.0  1074  
Loyal Customers      128356   817    155.0  124915.0  2846.0  2324964.0   817  
Need Attention         8555   205     98.0   11582.0   856.0   175520.0   205  
New Customers           432    57     42.0  168472.0  3619.0   206266.0    57  
Potential Loyalists   17190   492     69.0   26879.0   915.0   450420.0   492  
Promising               869   112     21.0    6749.0   429.0    48097.0   112